Data from https://tidesandcurrents.noaa.gov/waterlevels.html?id=9414290&units=standard&bdate=20190701&edate=20190801&timezone=GMT&datum=MLLW&interval=6&action=data

In [1]:
import math
import datetime
import pytz
import glob
import functools
import operator
import numpy
import pandas
import matplotlib.pyplot
import matplotlib.pylab
import seaborn
import sklearn.linear_model
import sklearn.metrics
import vtreat.cross_plan

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
date_fmt = '%Y/%m/%d %H:%M'
tz = pytz.utc

def parse_date(dtstr):
    d0 = datetime.datetime.strptime(dtstr, date_fmt)
    return d0.replace(tzinfo=tz)
    
base_date_time = datetime.datetime(2001, 1, 1, tzinfo=tz)

In [3]:
print("TZ NAME: {tz}".format(tz=base_date_time.tzname()))

TZ NAME: UTC


In [4]:
print(base_date_time)

2001-01-01 00:00:00+00:00


In [5]:
na_values = [ '', '-' ]
files = [f for f in glob.glob("tide_data/*.csv", recursive=False)]
files.sort()
tides = [pandas.read_csv(f, na_values=na_values) for f in files]
tides = pandas.concat(tides, axis=0)
tides.reset_index(inplace=True, drop=True)
tides.head()

,Date,Time (GMT),Predicted (ft),Preliminary (ft),Verified (ft)
0,2017/01/01,00:00,1.849,NaN,2.12
1,2017/01/01,00:06,1.695,NaN,1.97
2,2017/01/01,00:12,1.543,NaN,1.88
3,2017/01/01,00:18,1.393,NaN,1.78
4,2017/01/01,00:24,1.247,NaN,1.66


In [6]:
d0 = parse_date('2001/01/01 00:00')
(d0 - base_date_time).total_seconds()

0.0

In [7]:
print("TZ NAME: {tz}".format(tz=d0.tzname()))

TZ NAME: UTC


In [8]:
tides['dt'] = [parse_date(tides['Date'][i] + ' ' + tides['Time (GMT)'][i]) for i in range(tides.shape[0])]
tides['dts'] = [(t - base_date_time).total_seconds() for t in tides['dt']]

In [9]:
tides['tide feet'] = tides['Verified (ft)'].copy()
null_posns = pandas.isnull(tides['tide feet'])
tides.loc[null_posns, 'tide feet'] = tides.loc[null_posns, 'Preliminary (ft)']

In [10]:
tides.head()

,Date,Time (GMT),Predicted (ft),Preliminary (ft),Verified (ft),dt,dts,tide feet
0,2017/01/01,00:00,1.849,NaN,2.12,2017-01-01 00:00:00+00:00,504921600.0,2.12
1,2017/01/01,00:06,1.695,NaN,1.97,2017-01-01 00:06:00+00:00,504921960.0,1.97
2,2017/01/01,00:12,1.543,NaN,1.88,2017-01-01 00:12:00+00:00,504922320.0,1.88
3,2017/01/01,00:18,1.393,NaN,1.78,2017-01-01 00:18:00+00:00,504922680.0,1.78
4,2017/01/01,00:24,1.247,NaN,1.66,2017-01-01 00:24:00+00:00,504923040.0,1.66


In [11]:
numpy.mean(tides['tide feet'])

3.354944709837226

In [12]:
deltas = [tides['dts'][i+1] - tides['dts'][i] for i in range(tides.shape[0]-1)]

In [13]:
max(deltas)

360.0

In [14]:
min(deltas)

360.0

In [15]:
tides.to_pickle('tides.pickle.gz')